
#  <br/><br/><br/> An integrated framework for evaluating recommender engines using the developed package

### *Fan Xia*

#### Last edited on 06/2023 by Fan X.

### Introduction
#### An integrated framework for evaluating various recommendation algorithms is developed using the developed library. The developed package includes metric, dataset, evaluator, comparer and deployer modules. Here, I use MovieLens latest small dataset to test the package.

### Import required packages

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

### Data preparation
#### Data preparation includes reading raw data from disk that are used for model building, checking missing data, cleanning up duplicated and abnormal data and creating cleaned data for recommender system.

In [3]:
# Load clean data from disk for comparison
filename = '../data/ml_small/ratings_c1.csv'
data = pd.read_csv(filename)

data

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [4]:
# Build popularity rankings data frame
ranking_data = pd.DataFrame(data['movieId'].value_counts().sort_index(ascending=True)).reset_index(drop=False)
ranking_data['rank'] = ranking_data.index + 1
ranking_data.set_index("index", drop=True, inplace=True)
ranking_data = ranking_data.drop('movieId', axis=1)

ranking_data

,rank
index,
1,1
2,2
3,3
4,4
5,5
...,...
161944,9062
162376,9063
162542,9064


In [5]:
# Build popularity ranking dictionary
rankings = ranking_data.T.to_dict('list')
for k, v in rankings.items():
    rankings[k] = int(v[0])
rankings[162672]

9065

In [6]:
# Build catalog items as type of list
catalog = data['movieId'].unique().tolist()
catalog[0]

31

In [7]:
# Build item ID to item relations
import csv

table = {}
filename = '../data/ml_small/movies.csv'
with open(filename, newline='', encoding='ISO-8859-1') as file:
    reader = csv.reader(file)
    next(reader)
    for line in reader:
        iid = int(line[0])
        tag = line[1]
        table[iid] = tag

### Load data into Surprise library

In [8]:
from surprise import Dataset
from surprise import Reader

min_rating = data['rating'].min()
max_rating = data['rating'].max()
print("Set rating scale to: ", min_rating, max_rating)

reader = Reader(rating_scale=(min_rating, max_rating))
ratings = Dataset.load_from_df(data, reader)

Set rating scale to:  0.5 5.0


### Compare recommender engines using *mypkg* package

In [9]:
# Declare the local path of the package
import sys
sys.path.append(r'../src/mypkg')
# Load the module
from comparer import comparer

In [10]:
import random
np.random.seed(0)
random.seed(0)

In [11]:
# Construct a comparer class
cmp = comparer(data=ratings, rankings=rankings, catalog=catalog)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
from surprise import NormalPredictor
from surprise import SVD
from bprs import BPR

# Reset algorithms queue
cmp.reset()

# Add a random recommender
random = NormalPredictor()
cmp.addAlgorithm(random, "Random")

# Add an SVD recommender
svd = SVD(random_state=10)
cmp.addAlgorithm(svd, "SVD")

# Add a BPR recommender
bpr = BPR(verbose=False)
cmp.addAlgorithm(bpr, "BPR")

In [14]:
# Enable top-n ranking metrics
metrics = ['HR', 'MRR', 'Cov', 'Div', 'Nov']
cmp.doTopN(n=10, enable=True)

In [15]:
# Compare performance of recommenders
result = cmp.compare(metrics=metrics)

compare: evaluating Random...
eval: hit-rate and ranking metrics...
eval: coverage and novelty metrics...
Computing the cosine similarity matrix...
Done computing similarity matrix.
eval: complete [Random]
compare: evaluating SVD...
eval: hit-rate and ranking metrics...
eval: coverage and novelty metrics...
Computing the cosine similarity matrix...
Done computing similarity matrix.
eval: complete [SVD]
compare: evaluating BPR...
bpr: training...


2023-06-19 11:52:53.782273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████████████████████████| 1/1 [00:06<00:00,  6.46s/epoch, loss=0.609]


eval: hit-rate and ranking metrics...
bpr: training...


100%|██████████████████████████████| 1/1 [00:06<00:00,  6.81s/epoch, loss=0.615]


eval: coverage and novelty metrics...
Computing the cosine similarity matrix...
Done computing similarity matrix.
eval: complete [BPR]


In [16]:
# Summarize performance of recommenders
cmp.summary(result)

Engine    HR        MRR       Cov       Div       Nov       
Random    0.0089    0.0019    0.0312    0.071     951.2088  
SVD       0.0313    0.0114    0.0424    0.0462    1646.8794 
BPR       0.1013    0.042     0.0139    0.0347    1034.0256 


In [17]:
# Specify a customer ID
user = 85

In [18]:
# Return top-n recommendations for this user
cmp.sampleTopN(table, user)


sample: selecting recommender algorithm:  Random
sample: top-n recommendations for user: 85
Sleepers (1996)
Gandhi (1982)
First Knight (1995)
Houseguest (1994)
Like Water for Chocolate (Como agua para chocolate) (1992)
Harold and Maude (1971)
Blade Runner (1982)
Vertigo (1958)
Bedknobs and Broomsticks (1971)
Goodfellas (1990)

sample: selecting recommender algorithm:  SVD
sample: top-n recommendations for user: 85
All About Eve (1950)
Star Wars: Episode V - The Empire Strikes Back (1980)
Fight Club (1999)
Cool Hand Luke (1967)
Goodfellas (1990)
Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009)
General, The (1926)
African Queen, The (1951)
Graduate, The (1967)
When We Were Kings (1996)

sample: selecting recommender algorithm:  BPR
bpr: training...


100%|██████████████████████████████| 1/1 [00:05<00:00,  5.99s/epoch, loss=0.605]


sample: top-n recommendations for user: 85
Star Wars: Episode IV - A New Hope (1977)
Matrix, The (1999)
Seven (a.k.a. Se7en) (1995)
Star Wars: Episode VI - Return of the Jedi (1983)
Fight Club (1999)
Lord of the Rings: The Fellowship of the Ring, The (2001)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Apollo 13 (1995)
Usual Suspects, The (1995)
Sixth Sense, The (1999)


In [20]:
# Deploy SVD model for recommendation
from deployer import deployer
dpr = deployer(bpr, "BPR")
dpr.build(ratings)
top = dpr.get(table, user)

bpr: training...


100%|██████████████████████████████| 1/1 [00:06<00:00,  6.58s/epoch, loss=0.617]


BPR recommendations for user: 85
Star Wars: Episode IV - A New Hope (1977)
Seven (a.k.a. Se7en) (1995)
Matrix, The (1999)
Toy Story (1995)
Fargo (1996)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Usual Suspects, The (1995)
Men in Black (a.k.a. MIB) (1997)
Star Wars: Episode V - The Empire Strikes Back (1980)
Godfather, The (1972)


### Supplementary function